In [2]:
import numpy  as np
import pandas as pd
from torch.optim import Adam
from torch.nn import MSELoss,BCELoss
from torch.nn.functional import binary_cross_entropy
from torch import LongTensor
import torch
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

In [20]:
from Dataset import *
from torch.utils.data import DataLoader
np.random.seed(2022)
train = DebiasDataset(datatype='train')
test = DebiasDataset(datatype='test')

train_loader = DataLoader(dataset=train,batch_size=64,shuffle=True)
test_loader = DataLoader(dataset=test,batch_size=4640,shuffle=True)

In [24]:
#’---------MF----------------‘

from MF import *

model = MF(290,300)
optimizer = Adam(model.parameters(),lr = 0.01,weight_decay=0)
func_loss = BCELoss()
early_stop = 0
last_loss = 1e9

for epoch in tqdm(range(1000)):
    epoch_loss = 0
    for step,((user_idx,item_idx),rates) in enumerate(train_loader):

        rates_pre = model(user_idx,item_idx)

        loss = func_loss(rates_pre,rates.float())

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        epoch_loss += loss.detach().numpy()

    relative_loss_div = (last_loss-epoch_loss)/(last_loss+1e-10)
    if  relative_loss_div < 1e-6:
        if early_stop > 5:
            print("[MF-IPS] epoch:{}, xent:{}".format(epoch, epoch_loss))
            break
        early_stop += 1
    last_loss = epoch_loss


#-------------test------------------
mse = 0.0
mse_func = lambda x,y: torch.mean((x-y)**2)
with torch.no_grad():
    for step,((user_idx,item_idx),rates) in enumerate(test_loader):

        rates_pre = model(user_idx,item_idx)
        mse = mse_func(rates_pre,rates.float())
        auc = roc_auc_score(rates,rates_pre)
        print('mse is {}'.format(mse))
        print('auc is {}'.format(auc))
    

 30%|██▉       | 299/1000 [00:20<00:49, 14.30it/s]

[MF-IPS] epoch:299, xent:2.104789218471126e-07
mse is 0.40332332253456116
auc is 0.6191999591697541


In [26]:
#----MF-IPS  IPS from MAR--------'
from Propensity import *
from MF import *
#cauculate Inverse Propensity Score

config = {'num_users':290,'num_items':300,'propensity_model':'popularity'}

propensity = Propensity(config)
data = propensity.get_data()
propensity.fit(data)
score = propensity.predict(data)


model = MF(290,300)
optimizer = Adam(model.parameters(),lr = 0.01,weight_decay=0)
func_loss = MSELoss(reduction= 'none')
early_stop = 0

for epoch in tqdm(range(1000)):
    epoch_loss = 0
    for step,((user_idx,item_idx),rates) in enumerate(train_loader):

        rates_pre = model(user_idx,item_idx)

        prop_score = torch.FloatTensor(score[item_idx])

        loss = binary_cross_entropy(rates_pre,rates.float(),weight=prop_score)
        # loss = torch.mean(loss)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        epoch_loss += loss.detach().numpy()

    relative_loss_div = (last_loss-epoch_loss)/(last_loss+1e-10)
    if  relative_loss_div < 1e-6:
        if early_stop > 5:
            print("[MF-IPS] epoch:{}, xent:{}".format(epoch, epoch_loss))
            break
        early_stop += 1
    last_loss = epoch_loss



#-------------test------------------
mse = 0.0
mse_func = lambda x,y: torch.mean((x-y)**2)
with torch.no_grad():
    for step,((user_idx,item_idx),rates) in enumerate(test_loader):

        rates_pre = model(user_idx,item_idx)
        mse = mse_func(rates_pre,rates.float())
        auc = roc_auc_score(rates,rates_pre)
        print('mse is {}'.format(mse))
        print('auc is {}'.format(auc))


 29%|██▉       | 293/1000 [00:26<01:04, 10.91it/s]

[MF-IPS] epoch:293, xent:1.5409365641261275e-06
mse is 0.39592549204826355
auc is 0.6215898431670042


In [23]:
# --- MF_IPS  IPS from MNAR----
from Propensity import *
from MF import *
#cauculate Inverse Prop
# ensity Score

config = {'num_users':290,'num_items':300,'propensity_model':'popularity'}

propensity = Propensity(config)
data = propensity.get_data(data_type='train')
propensity.fit(data)
score = propensity.predict(data)


model = MF(290,300)
optimizer = Adam(model.parameters(),lr = 0.01,weight_decay=0)
func_loss = MSELoss(reduction= 'none')
early_stop = 0

for epoch in tqdm(range(1000)):
    # if epoch != 0:
    #     data = model.prediect().numpy()
    #     data = propensity.get_data(datas=data)
    # propensity.fit(data)
    # score = propensity.predict(data)
    model.train()
    epoch_loss = 0

    for step,((user_idx,item_idx),rates) in enumerate(train_loader):

        rates_pre = model(user_idx,item_idx)

        prop_score = torch.FloatTensor(score[item_idx])

        loss = binary_cross_entropy(rates_pre,rates.float(),weight=prop_score)
        # loss = torch.mean(loss)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        epoch_loss += loss.detach().numpy()

    relative_loss_div = (last_loss-epoch_loss)/(last_loss+1e-10)
    if  relative_loss_div < 1e-6:
        if early_stop > 5:
            print("[MF-IPS] epoch:{}, xent:{}".format(epoch, epoch_loss))
            break
        early_stop += 1
    last_loss = epoch_loss


#-------------test------------------
mse = 0.0
mse_func = lambda x,y: torch.mean((x-y)**2)
with torch.no_grad():
    for step,((user_idx,item_idx),rates) in enumerate(test_loader):

        rates_pre = model(user_idx,item_idx)
        mse = mse_func(rates_pre,rates.float())
        auc = roc_auc_score(rates,rates_pre)
        print('mse is {}'.format(mse))
        print('auc is {}'.format(auc))


 28%|██▊       | 284/1000 [00:24<01:01, 11.71it/s]

[MF-IPS] epoch:284, xent:3.2628204795770444e-06
mse is 0.4008309245109558
auc is 0.6231375066793797
